主要参考了两篇博客：
- http://www.matrix67.com/blog/archives/5044
- http://spaces.ac.cn/archives/3491/

主要的思想是利用统计信息进行过滤，用到的统计信息包括：
- 词频
- 单词内部的凝合程度
- 单词上下文的信息熵

In [2]:
cd ../Downloads/

C:\Users\Jason\Downloads


In [1]:
import re
from collections import Counter
import numpy as np

In [3]:
data = open("./天龙八部精校版.txt").read()

In [4]:
len(data)

1258690

In [8]:
split_set = set([u'】',u'【', u'，', u'\n', u'。', u'、', u'：', u'(', u')', u'[', u']', u'.', u',', u' ', u'\u3000', u'”', u'“', u'？', u'?', u'！', u'‘', u'’', u'…'])

In [9]:
min_count = 10 # 最小频数
min_support = 30 # 最小的凝合程度
min_ms = 3 # 最小的信息熵

In [10]:
# 最好是按照split set把文本分割成短句子，然后在短句子基础上再做新词发现
# 可能效果会更好，但是在大文本的情况下，个人觉得直接去除标点字符也会取得不错的效果
for i in split_set:
    data=data.replace(i,'')

In [11]:
total = len(data)

In [12]:
gram_1 = Counter(list(data))

In [13]:
gram_1 = dict(gram_1)

In [28]:
grams = [gram_1]
max_gram = 4
ngram_pattern = {2:'(..)', 3:'(...)', 4:'(....)', 5:'(.....)', 6:'(......)', 7:'(.......)'}

In [29]:
remains = [set(list(gram_1.keys()))]

In [30]:
for n in range(2,max_gram+1):
    print("genetate {0}-gram".format(n))
    all_xgram = []
    for i in range(n):
        all_xgram = all_xgram + re.findall(ngram_pattern[n],data[i:])
    gram_x = dict(Counter(all_xgram))
    grams.append(gram_x)
    print("raw {0}-gram count {1}".format(n,len(gram_x)))
    gram_x = {k:v for k,v in gram_x.items() if v > min_count}
    print("after filter: {0}".format(len(gram_x)))
    
    #凝合程度
    remain = []
    for word in gram_x:
        for k in range(1,n):
            left = word[:k]
            left_count = grams[len(left)-1][left]
            right = word[k:]
            right_count = grams[len(right)-1][right]
            a = total * gram_x[word] / left_count / right_count
            if a >= min_support:
                remain.append(word)
    remains.append(set(remain))
    print("{0}-gram candidates: {1}".format(n,len(remains[-1])))

genetate 2-gram
raw 2-gram count 218814
after filter: 14554
2-gram candidates: 3611
genetate 3-gram
raw 3-gram count 624263
after filter: 6632
3-gram candidates: 5563
genetate 4-gram
raw 4-gram count 865724
after filter: 1879
4-gram candidates: 1869


In [31]:
def info_entropy(array):
    array = np.asarray(array)
    return np.sum(-(array / array.sum() ) * (np.log(array / array.sum())))    

In [47]:
final_words = []
for n in range(2,max_gram+1):
    context=[]
    for i in range(n):
        context = context + re.findall('(.){0}(.)'.format(ngram_pattern[n]),data[i:])
    mutual_info = {}
    remain_word = remains[n-1]
    print("get context info")
    for left,word,right in context:
        if word not in remain_word:
            continue
        if word not in mutual_info:
            mutual_info[word] = {'left':{},'right':{}}
        if left not in mutual_info[word]['left']:
            mutual_info[word]['left'][left] = 0
        if right not in mutual_info[word]['right']:
            mutual_info[word]['right'][right] = 0
        mutual_info[word]['left'][left] += 1
        mutual_info[word]['right'][right] += 1
    print("filter based on entropy")
    
    words = []
    for word in remain_word:
        if word not in mutual_info:
            continue
        left = np.array(list(mutual_info[word]['left'].values()))
        right = np.array(list(mutual_info[word]['right'].values()))
        ls = info_entropy(left)
        rs = info_entropy(right)
        ms = min(ls,rs)
        if ms >= min_ms:
            words.append(word)
    print("{0}-gram words: {1}".format(n,len(words)))
    final_words = final_words + words

get context info
filter based on entropy
2-gram words: 244
get context info
filter based on entropy
3-gram words: 188
get context info
filter based on entropy
4-gram words: 46


In [48]:
words = [(w,grams[len(w)-1][w]) for w in final_words]
words = sorted(words,key=lambda w: w[1],reverse=True)

In [50]:
for word,count in words[:100]:
    print(word,count)

段誉 3375
萧峰 1787
虚竹 1638
自己 1638
什么 1595
阿紫 1151
乔峰 1131
武功 1073
甚么 1034
阿朱 986
姑娘 983
慕容复 927
王语嫣 859
师父 818
咱们 818
如何 777
段正淳 758
木婉清 735
如此 699
突然 654
心想 606
大理 598
帮主 592
童姥 583
弟子 580
不敢 578
鸠摩智 564
丐帮 562
怎么 541
游坦之 534
众人 526
内力 509
倘若 505
南海鳄神 485
之间 474
脸上 470
段誉道 468
兄弟 467
原来 464
这些 455
钟灵 442
丁春秋 437
天下 434
跟着 422
当真 415
说话 415
爹爹 412
包不同 410
伸手 405
登时 402
功夫 400
和尚 397
今日 393
少林寺 388
没有 372
保定帝 341
眼见 334
左手 333
当即 331
英雄 327
师兄 327
有什么 327
虽然 324
双手 319
阿碧 307
这般 306
段延庆 302
乌老大 302
厉害 301
马夫人 300
之极 294
只怕 292
右手 282
十分 277
便即 277
不由得 277
哪里 276
一阵 272
只听得 271
玄难 266
似乎 264
不肯 264
方丈 264
对方 260
胸口 253
居然 251
明白 251
中原 249
小僧 247
虚竹道 245
轻轻 240
无法 240
了出来 240
玄慈 239
仍是 238
此刻 238
王夫人 238
※※※ 236
那少女 234
只觉 232


### 思考
可以发现这个方法可以找到诸如人名等一些新词，但是还是存在一些问题，比如像“段誉道”，“了出来”这样一些bad cases。自己想了一些解决方法：
- 通过对不同n-gram设置不同的threshold
- 仅仅通过这3个feature是不够的，看论文的话一般还会利用语法的一些feature